# Milestone 2
 The following Test code is about to set functions for our Milestone 2 create a chat with only the data on our database

In [12]:
import os
from dotenv import load_dotenv as envFile
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [13]:
envFile()

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

db = Chroma(
    persist_directory= ".\chroma_db", embedding_function=embeddings
)

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash", temperature = 0
)

<>:6: SyntaxWarning: invalid escape sequence '\c'
<>:6: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_1059816/2039635687.py:6: SyntaxWarning: invalid escape sequence '\c'
  persist_directory= ".\chroma_db", embedding_function=embeddings


In [14]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.75, "k": 3}
)

In [15]:
systemPrompt = (
    "Eres un asistente que SOLO responde basándose en el contexto proporcionado. "
    "Para cada afirmación que hagas, debes indicar el título de la película del contexto. "
    "Si la información no está en el texto de abajo, di textualmente: 'No tengo esa información'.\n\n"
    "Contexto:\n"
    "{context}"
)

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", systemPrompt),
        ("human","{input}"),
    ]
)

In [17]:

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)



In [18]:
while True:
    pregunta = input("\nPregunta: ")
    if pregunta.lower() in ["salir", "q"]: break

    # Invocamos la cadena
    # LangChain intentará buscar. Si el threshold filtra todo, 'context' estará vacío.
    response = rag_chain.invoke({"input": pregunta})
    
    # VERIFICACIÓN MANUAL DE CONTEXTO
    if not response["context"]:
        print("🤖 Respuesta: No encontré nada relacionado en tu base de datos (Filtro de Similitud).")
    else:
        print(f"🤖 Respuesta: {response['answer']}")
        print("\n--- Evidencia Utilizada ---")
        for doc in response["context"]:
            print(f"📄 Fuente: {doc.metadata.get('titulo', 'Sin título')} (Similitud aceptada)")

No relevant docs were retrieved using the relevance score threshold 0.75


🤖 Respuesta: No encontré nada relacionado en tu base de datos (Filtro de Similitud).


No relevant docs were retrieved using the relevance score threshold 0.75


🤖 Respuesta: No encontré nada relacionado en tu base de datos (Filtro de Similitud).


No relevant docs were retrieved using the relevance score threshold 0.75


KeyboardInterrupt: 

In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

load_dotenv()

# 1. Cargar Base de Datos
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

while True:
    query = input("\nPregunta de prueba: ")
    if query in ["salir", "exit"]: break

    # Buscamos con puntajes (k=3)
    results = db.similarity_search_with_relevance_scores(query, k=3)

    if not results:
        print("⚠️ No se encontraron resultados (Puntaje muy bajo o DB vacía)")
    else:
        print(f"\nResultados para '{query}':")
        for doc, score in results:
            print(f"   📊 Puntaje: {score:.4f} | Película: {doc.metadata.get('Titulo', 'Sin título')}")
            print(f"      Fragmento: {doc.page_content[:100]}...")

    print("\n💡 CONCLUSIÓN:")
    if results and results[0][1] > 0.5:
        print("   ✅ Pasaría el filtro actual (0.75)")
    else:
        print("   ❌ Sería bloqueado por el filtro actual (0.75)")

/tmp/ipykernel_1155633/2214420860.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)


🧪 MODO DE CALIBRACIÓN
Escribe una pregunta para ver qué tan parecida es a tus documentos (0.0 a 1.0)

Resultados para 'Star Wars':
   📊 Puntaje: 0.5719 | Película: Sin título
      Fragmento: Título: Starwars: Goretech. Trama: No summary...
   📊 Puntaje: 0.5673 | Película: Sin título
      Fragmento: Título: Star Wars: Episode IX - The Rise of Skywal...
   📊 Puntaje: 0.5586 | Película: Sin título
      Fragmento: Título: Star Wars: Episode VII - The Force Awakens...

💡 CONCLUSIÓN:
   ✅ Pasaría el filtro actual (0.75)

Resultados para 'Lord of the rings':
   📊 Puntaje: 0.6100 | Película: Sin título
      Fragmento: Título: The Lord of the Rings. Trama: The Fellowsh...
   📊 Puntaje: 0.5857 | Película: Sin título
      Fragmento: Título: The Lord of the Rings Symphony. Trama: A f...
   📊 Puntaje: 0.5767 | Película: Sin título
      Fragmento: Título: The Lord of the Rings: The Return of the K...

💡 CONCLUSIÓN:
   ✅ Pasaría el filtro actual (0.75)

Resultados para 'Titanic':
   📊 Puntaje: 

GoogleGenerativeAIError: Error embedding content (INVALID_ARGUMENT): 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'The text content is empty.', 'status': 'INVALID_ARGUMENT'}}